In [31]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES, DES, ARC4, PKCS1_OAEP
from Crypto.Random import get_random_bytes

In [32]:
class Nigma:
    def __init__(self):
        pass

    def makeEnvelope(self, message, sessionKey, initializationVector, operationMode, methodCrypto):
        if methodCrypto == "AES":
            try:
                with open("./envelope/output/cryptogram.bin", "wb") as archive:
                    archive.write(self.encryptAES(message, sessionKey, initializationVector, operationMode))
                with open("./envelope/output/encryptSessionKey.bin", "wb") as archive:
                    archive.write(self.encryptSessionKey(sessionKey))
                return True
            except:
                return False
        elif methodCrypto == "DES":
            pass
        elif methodCrypto == "RC4":
            pass
        else:
            return False

    def generateRandomKey(self, sizeKey):
        return get_random_bytes(sizeKey)
    
    def generatePairKeys(self, sizeKey=1024):
        key = RSA.generate(sizeKey)
        try:
            with open("./openEnvelope/input/private.pem", "wb") as archive:
                archive.write(key.export_key())
            with open("./envelope/input/publickey.pem", "wb") as archive:
                archive.write(key.publickey().export_key())
            return True
        except:
            return False

    def encryptSessionKey(self, sessionKey:bytes) -> bytes:
        with open("./envelope/input/publickey.pem", "rb") as archive:
            recipient_key = RSA.import_key(archive.read())
            cipher_rsa = PKCS1_OAEP.new(recipient_key)
            return cipher_rsa.encrypt(sessionKey)
        
    def decryptSessionKey(self, encSessionKey):
        with open("./openEnvelope/input/private.pem") as archive:
            privateKey = RSA.import_key(archive.read())
            cipherRSA = PKCS1_OAEP.new(privateKey)
            try:
                with open("./openEnvelope/process/decryptSessionKey.bin", "wb") as archive:
                    archive.write(cipherRSA.decrypt(encSessionKey))
                return True
            except:
                return False

    def encryptAES(self, message, key, initializationVector, modeOperation):
        if modeOperation == AES.MODE_CBC:
            cipher = AES.new(key, modeOperation, initializationVector)
            sizeBlock = AES.block_size
            messagePadded = message + (sizeBlock - len(message) % sizeBlock) * b"\x00"
            return cipher.encrypt(messagePadded)
        elif modeOperation == AES.MODE_ECB:
            cipher = AES.new(key, modeOperation)

    
    def encryptDES(self, message, key, modeOperation):
        if modeOperation == 0:
            pass
        elif modeOperation == 1:
            pass
        elif modeOperation == 2:
            pass

    def encryptRC4(self, message, key, modeOperation):
        if modeOperation == 0:
            pass
        elif modeOperation == 1:
            pass
        elif modeOperation == 2:
            pass

    def decryptAES(self, key, initializationVector, modeOperation):
        if modeOperation == AES.MODE_CBC:
            with open("./envelope/output/cryptogram.bin", "rb") as archive:
                cryptogram = archive.read()
            cipher = AES.new(key, modeOperation, initializationVector)
            textLight = cipher.decrypt(cryptogram)
            text = textLight.rstrip(b"\x00")
            with open("./openEnvelope/output/text.txt", "wb") as archive:
                archive.write(text)
        elif modeOperation == AES.MODE_ECB:
            pass
    
    def decryptDES(self,cryptogram, key, tag, modeOperation):
        pass
    
    def decryptRC4(self,cryptogram, key, tag, modeOperation):
        pass

    def __str__(self):
        return "Hi, I'm Nigma!"

Debug Envelope

In [33]:
def main():
    maquina = Nigma()
    sessionKey = maquina.generateRandomKey(16)
    print(sessionKey)
    initializationVector = maquina.generateRandomKey(16)
    maquina.generatePairKeys(1024)

    with open("./envelope/input/message.txt", "rb") as archive:
        message = archive.read()

    maquina.makeEnvelope(message, sessionKey, initializationVector, AES.MODE_CBC, "AES")

    with open("./envelope/output/encryptSessionKey.bin", "rb") as archive:
        encryptSessionKey = archive.read()

    maquina.decryptSessionKey(encryptSessionKey)

    with open("./openEnvelope/process/decryptSessionKey.bin", "rb") as archive:
        sessionKey = archive.read()
    
    maquina.decryptAES(sessionKey, initializationVector, AES.MODE_CBC)
    
    
if __name__ == "__main__":
    main()

b'k\xfe!\x9b\t\x97Ta\x88\x01=Fu0\xd5\xd7'
